In [ ]:
import os
import sys
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')
import numpy as np
import polars as pl
import pandas as pd
import plotly.graph_objects as go
pd.options.display.max_rows = None
pd.options.display.max_columns = None
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import kaggle_evaluation
import kaggle_evaluation.mcts_inference_server
import lightgbm as lgb
from catboost import CatBoostRegressor
import kaggle_evaluation.mcts_inference_server
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error as mse

In [ ]:
class CFG:
    
    importances_path = Path('/kaggle/input/mcts-gbdt-select-200-features/importances.csv')    
    train_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
    batch_size = 65536

    early_stop = 500
    n_splits = 8
    color = '#C9A9A6'
    
    lgb_w = 0.80
    lgb_p = {
        'objective': 'regression',
        'min_child_samples': 24,
        'num_iterations': 20000,
        'learning_rate': 0.07,
        'extra_trees': True,
        'reg_lambda': 0.8,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'rmse',
        'device': 'cpu',
        'max_depth': 24,
        'max_bin': 128,
        'verbose': -1,
        'seed': 42
    }
    
    ctb_w = 0.30
    ctb_p = {
        'loss_function': 'RMSE',
        'learning_rate': 0.03,
        'num_trees': 20000,
        'random_state': 42,
        'task_type': 'CPU',
        'reg_lambda': 0.8,
        'depth': 8
    }

In [ ]:
class FE:
    
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
    def drop_cols(self, df, bad_cols=None): # bad_cols must be provided when processing the test data
        
        # Define redundant columns for model development
        cols = ['Id', 
                'LudRules', 
                'EnglishRules',
                'num_wins_agent1',
                'num_draws_agent1',
                'num_losses_agent1']
        
        df = df.drop([col for col in cols if col in df.columns])
        
        # Select and drop columns with 100% null values
        df = df.drop([col for col in df.columns if df.select(pl.col(col).null_count()).item() == df.height])
        
        # Select (if not provided) and drop columns with only one unique value
        bad_cols = [col for col in df.columns if df.select(pl.col(col).n_unique()).item() == 1] if bad_cols is None else bad_cols
        df = df.drop(bad_cols)
        
        return df, bad_cols
    
    def cast_datatypes(self, df):
        
        # Set datatype for categorical columns
        cat_cols = ['GameRulesetName', 'agent1', 'agent2']
        df = df.with_columns([pl.col(col).cast(pl.String) for col in cat_cols])   
        
        # Find numeric columns
        for col in df.columns:
            if col not in cat_cols:
            
                # Set datatype for a numeric column as per the datatype of the first non-null item
                val = df.select(pl.col(col).drop_nulls().first()).item()
                df = df.with_columns(pl.col(col).cast(pl.Int16) if isinstance(val, int) else pl.col(col).cast(pl.Float32))   
            
        return df    
    
    def info(self, df):
        
        print(f'Shape: {df.shape}')   
        mem = df.estimated_size() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))
        
    def apply_fe(self, path):
        
        df = pl.read_csv(path, batch_size=self.batch_size)
        
        df, bad_cols = self.drop_cols(df)
        df = self.cast_datatypes(df)
        self.info(df)
        
        cat_cols = [col for col in df.columns if df[col].dtype == pl.String]
        
        return df, bad_cols, cat_cols

In [ ]:
fe = FE(CFG.batch_size)

In [ ]:
class MD:
    
    def __init__(self, 
                 importances_path, 
                 early_stop, 
                 n_splits, 
                 lgb_w, 
                 lgb_p, 
                 ctb_w, 
                 ctb_p, 
                 color):
        
        self.importances_path = importances_path
        self.early_stop = early_stop
        self.n_splits = n_splits
        self.lgb_w = lgb_w
        self.lgb_p = lgb_p
        self.ctb_w = ctb_w
        self.ctb_p = ctb_p
        self.color = color
        
    def plot_cv(self, fold_scores, title):
        
        fold_scores = [round(score, 3) for score in fold_scores]
        mean_score = round(np.mean(fold_scores), 3)
        std_score = round(np.std(fold_scores), 3)

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x = list(range(1, len(fold_scores) + 1)),
            y = fold_scores,
            mode = 'markers', 
            name = 'Fold Scores',
            marker = dict(size = 24, color=self.color, symbol='diamond'),
            text = [f'{score:.3f}' for score in fold_scores],
            hovertemplate = 'Fold %{x}: %{text}<extra></extra>',
            hoverlabel=dict(font=dict(size=16))  
        ))

        fig.add_trace(go.Scatter(
            x = [1, len(fold_scores)],
            y = [mean_score, mean_score],
            mode = 'lines',
            name = f'Mean: {mean_score:.3f}',
            line = dict(dash = 'dash', color = '#FFBF00'),
            hoverinfo = 'none'
        ))

        fig.update_layout(
            title = f'{title} | Cross-Validation RMSE Scores | Variation of CV scores: {mean_score} ± {std_score}',
            xaxis_title = 'Fold',
            yaxis_title = 'RMSE Score',
            plot_bgcolor = 'rgba(0,0,0,0)',
            paper_bgcolor = 'rgba(0,0,0,0)',
            xaxis = dict(
                gridcolor = 'lightgray',
                tickmode = 'linear',
                tick0 = 1,
                dtick = 1,
                range = [0.5, len(fold_scores) + 0.5]
            ),
            yaxis = dict(gridcolor = 'lightgray')
        )

        fig.show() 
        
    def train_model(self, data, cat_cols, title):
        
        importances = pd.read_csv(self.importances_path)

        for col in cat_cols:
            data[col] = data[col].astype('category')
        
        # Define features (X), label (y) and grouping column (group) for CV
        X = data.drop(['utility_agent1'], axis=1)
        y = data['utility_agent1']
        group = data['GameRulesetName']
        
        cv = GroupKFold(n_splits=self.n_splits)
        
        models, scores = [], []
        
        # Initialize out-of-fold predictions array
        oof_preds = np.zeros(len(X))
        
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y, group)):
            
            drop_features = importances['drop_features'].tolist()
                
            X_train, X_valid = X.iloc[train_index].drop(drop_features, axis=1), X.iloc[valid_index].drop(drop_features, axis=1)
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            print(f'Fold {fold+1} | {X_train.shape[0]:,} train rows | {X_valid.shape[0]:,} valid rows | {X_train.shape[1]} features')
                
            if title.startswith('LightGBM'):

                model = lgb.LGBMRegressor(**self.lgb_p)

                model.fit(X_train, y_train,
                          eval_set=[(X_valid, y_valid)],
                          eval_metric='rmse',
                          callbacks=[lgb.early_stopping(self.early_stop, verbose=0), lgb.log_evaluation(0)])
            
            elif title.startswith('CatBoost'):
            
                model = CatBoostRegressor(**self.ctb_p, verbose=0, cat_features=cat_cols)

                model.fit(X_train, y_train,
                          eval_set=(X_valid, y_valid),
                          early_stopping_rounds=self.early_stop, verbose=0)

            models.append(model)

            # Store out-of-fold predictions for this fold
            oof_preds[valid_index] = model.predict(X_valid)
            score = mse(y_valid, oof_preds[valid_index], squared=False)
            scores.append(score)
        
        self.plot_cv(scores, title)
        
        return models, oof_preds
    
    def inference(self, data, cat_cols, lgb_models, ctb_models, lgb_models_oof, ctb_models_oof):

        importances = pd.read_csv(self.importances_path)
            
        drop_features = importances['drop_features'].tolist()
        data = data.drop(drop_features, axis=1)

        for col in cat_cols:
            data[col] = data[col].astype('category')
                
        data['lgb_oof_preds'] = np.mean([model.predict(data) for model in lgb_models], axis=0)
        data['ctb_oof_preds'] = np.mean([model.predict(data) for model in ctb_models], axis=0)
        
        lgb_preds = np.mean([model.predict(data) for model in lgb_models_oof], axis=0)  
        ctb_preds = np.mean([model.predict(data) for model in ctb_models_oof], axis=0)    
        
        return lgb_preds * self.lgb_w + ctb_preds * self.ctb_w

In [ ]:
md = MD(CFG.importances_path, 
        CFG.early_stop, 
        CFG.n_splits, 
        CFG.lgb_w, 
        CFG.lgb_p, 
        CFG.ctb_w, 
        CFG.ctb_p, 
        CFG.color)

In [ ]:
def train_model():
    
    global bad_cols, cat_cols, lgb_models, ctb_models, lgb_models_oof, ctb_models_oof
    
    train, bad_cols, cat_cols = fe.apply_fe(CFG.train_path)
    train = train.to_pandas()
        
    lgb_models, lgb_oof_preds = md.train_model(train, cat_cols, title='LightGBM')
    ctb_models, ctb_oof_preds = md.train_model(train, cat_cols, title='CatBoost')
    
    train['lgb_oof_preds'] = lgb_oof_preds
    train['ctb_oof_preds'] = ctb_oof_preds
    
    lgb_models_oof, _ = md.train_model(train, cat_cols, title='LightGBM w/ OOF')
    ctb_models_oof, _ = md.train_model(train, cat_cols, title='CatBoost w/ OOF')

In [ ]:
counter = 0
def predict(test, submission):
    
    global counter
    
    if counter == 0:
        train_model() 
        
    counter += 1
    
    test, _ = fe.drop_cols(test, bad_cols)
    test = fe.cast_datatypes(test)
    test = test.to_pandas()
    
    return submission.with_columns(pl.Series('utility_agent1', md.inference(test,
                                                                            cat_cols, 
                                                                            lgb_models, 
                                                                            ctb_models, 
                                                                            lgb_models_oof, 
                                                                            ctb_models_oof)))

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )